In [5]:
%load_ext autoreload
%autoreload 2

import sys
import requests
import pandas as pd
import datetime as dt

import numpy as np
from velib.data import Velib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
Velib().get_stations_df()

,station_id,name,lat,lon,capacity,stationCode,rental_methods
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN
1,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,NaN
2,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD]
3,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN
4,100769544,Harpe - Saint-Germain,48.851519,2.343670,46,5001,[CREDITCARD]
...,...,...,...,...,...,...,...
1432,13373801574,Gare RER,48.896241,2.459576,0,31301,[CREDITCARD]
1433,13373848216,Simone Signoret - Paris,48.903601,2.465407,0,31303,NaN
1434,13373758036,Roland Garros - Jean Bouin,48.809461,2.285320,0,23204,NaN
1435,315022587,Malesherbes - Place de la Madeleine,48.870406,2.323244,67,8004,[CREDITCARD]


In [7]:
Velib().get_velib_df()

,stationCode,station_id,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,16107,213688169,2,2,"[{'mechanical': 0}, {'ebike': 2}]",32,32,1,1,1,1635016329
1,11104,516709288,3,3,"[{'mechanical': 1}, {'ebike': 2}]",16,16,1,1,1,1635016164
2,9020,36255,4,4,"[{'mechanical': 4}, {'ebike': 0}]",17,17,1,1,1,1635016217
3,12109,37815204,19,19,"[{'mechanical': 14}, {'ebike': 5}]",8,8,1,1,1,1635016422
4,5001,100769544,31,31,"[{'mechanical': 29}, {'ebike': 2}]",11,11,1,1,1,1635016404
...,...,...,...,...,...,...,...,...,...,...,...
1432,31301,13373801574,0,0,"[{'mechanical': 0}, {'ebike': 0}]",0,0,0,0,0,1634892730
1433,31303,13373848216,0,0,"[{'mechanical': 0}, {'ebike': 0}]",0,0,0,0,0,1634893195
1434,23204,13373758036,0,0,"[{'mechanical': 0}, {'ebike': 0}]",0,0,0,0,0,1634892200
1435,8004,315022587,52,52,"[{'mechanical': 48}, {'ebike': 4}]",15,15,1,1,1,1635016427


In [6]:
URL_VELIB_STATUS = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"
URL_VELIB_STATIONS = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json"

def get_velib_df():
    response = requests.get(URL_VELIB_STATUS)
    velib_df = (pd.DataFrame(response.json()["data"]["stations"])
                    .set_index("stationCode"))
    return velib_df

def get_stations_df():
    response = requests.get(URL_VELIB_STATIONS)
    stations_df = (pd.DataFrame(response.json()["data"]["stations"])
                   .set_index("station_id"))
    return stations_df


In [25]:
velib_df = get_velib_df()

In [26]:
velib_df.shape

(1432, 10)

In [37]:
velib_df.index.unique()

Index(['16107', '11104', '9020', '12109', '5001', '14014', '17026', '10013',
       '8026', '14111',
       ...
       '23203', '32012', '17127', '34012', '21302', '8004', '26008', '26009',
       '17126', '34005'],
      dtype='object', name='stationCode', length=1432)

In [21]:
def find_mecha(row):
    return row['num_bikes_available_types'][0].get('mechanical', np.nan)
def find_elec(row):
    return row['num_bikes_available_types'][1].get('ebike', np.nan)

In [28]:
velib_df['meca'] = velib_df.apply(find_mecha, axis = 1)
velib_df['elec'] = velib_df.apply(find_elec, axis = 1)
velib_df['park'] = velib_df['num_docks_available']
velib_df['percent'] = velib_df['num_bikes_available']/(velib_df['num_bikes_available'] + velib_df['num_docks_available'])

In [36]:
velib_df.percent.sum()

512.180518753291

In [23]:
velib_df = velib_df[['station_id', 'meca', 'elec', 'park']]

In [24]:
velib_df

,station_id,meca,elec,park
stationCode,,,,
16107,213688169,1,1,32
11104,516709288,2,3,14
9020,36255,10,3,8
12109,37815204,21,3,5
5001,100769544,14,3,27
...,...,...,...,...
8004,315022587,2,2,62
26008,11437761399,0,0,0
26009,11437961617,0,0,0
